In [1]:
import tensorflow as tf
import numpy as np
from data_input import get_sparse_data
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.utils import to_categorical
from classifiers import check_acc
from keras import regularizers
from keras.layers import LSTM, Flatten,Conv1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

[y_train,X_raw_train,X_train_sparse],[y_val,X_raw_val,X_val_sparse],[y_test,X_raw_test,X_test_sparse] = get_sparse_data()

#convert into one hot
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_val_hot = to_categorical(y_val)

input_shape = X_train_sparse.get_shape()
print("Trainig shape:",input_shape)

Using TensorFlow backend.


Trainig shape: (14221, 1797)


[2,
 1,
 2,
 0,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 0,
 0,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 1,
 1,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 0,
 2,
 1,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 2,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 2,
 1,
 0,
 0,
 2,
 2,
 1,


2
that player is a complete piece of sh ***.

[ 0.  0.  0. ...,  0.  0.  0.]


In [ ]:
model = Sequential()
model.add(Embedding(input_dim = input_shape[1],output_dim = embedding_vecor_length , input_length=input_shape[1]))
model.add(Conv1D(64, 3, border_mode='same'))
model.add(Conv1D(32, 3, border_mode='same'))
model.add(Conv1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

/home/johnnyhan654/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/johnnyhan654/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
  after removing the cwd from sys.path.
/home/johnnyhan654/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(16, 3, padding="same")`
  """


In [ ]:
# Train the model, iterating on the data in batches of 32 samples
hist = model.fit(X_train_sparse.toarray(), y_train_hot, epochs=20,
          validation_data=(X_val_sparse.toarray(),y_val_hot),
          batch_size= 1024
          ) 

(loss, accuracy) = model.evaluate(X_test_sparse.toarray(), y_test_hot)
print(accuracy)

Train on 14221 samples, validate on 7110 samples
Epoch 1/20
